# Natural Language Processing

## Neural Transition-Based Dependency Parsing

We will be implementing dependency parsing from scratch, in order to gain a deep understanding of dependency parser, following *A Fast and Accurate Dependency Parser using Neural Networks (Chen and Manning 2014)* - https://aclanthology.org/D14-1082.pdf

We will be implementing a neural dependency parser with the goal of maximizing the performance on the UAS (Unlabeled Attachment Score) metric.

A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between head words, and words which modify those heads.  There are multiple types of dependency parsers, including transition-based parsers, graph-based parsers, and feature-based parsers.  We shall implement the transition-based parser, which incrementally builds up a parse one step at a time.  At every step, it maintains a partial parse, which is represented as follows:

- A **stack** of words that are currently being processed
- A **buffer** of words yet to be processed.
- A **list of dependencies** predicted by the parser

Initially, the stack only contains ROOT, the dependencies list is empty, and the buffer contains all words of the sentence in order. At each step, the parser applies a tranistion to the partial parse until its buffer is empty and the stack size is 1.  The following transitions can be applied:

- $\texttt{SHIFT}$: removes the first word from the buffer and pushes it onto the stack.
- $\texttt{LEFTARC}$: marks the second (second most recently aded) item on the stack as a dependent of the first item and removes the second item from the stack, adding a *first_word* $\rightarrow$ *second_word* dependency to the dependeny list.
- $\texttt{RIGHTARC}$: marks the first (second msot recently aded) item on the stack as a dependent of the second item and removes the first item from the stack, adding a *second_word* $\rightarrow$ *first_word* dependency to the dependeny list.

On each step, your parser will decide among the three transitions using a neural network classifier.

For parsing the sentence *He has good control*, the dependency tree for the sentence is shown below.

<img src = "../../../figures/dependency.png" width=900>

In [1]:
import sys
import numpy as np
import time
import os
import logging
from collections import Counter
from datetime import datetime
import math

from tqdm import tqdm  #gimmick for progressbar when you train
import pickle #saving and loading models

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim

To make the dependency parser, there will be lot of component.

1. The transition parsing function - indicate what SHIFT, LEFT-ARC, and RIGHT-ARC do
2. Loading the data (conll) - teach you how to load the data
3. The actual parser - combine everything into the real parser
4. The embedding lookup - embedding (i coded this for you because i think you guys are already quite good at this....)
5. The deep learning model - maybe you are most interested (but you may be dissapointed )
6. The training and evaluation - also I coded this for you because there's little to learn here....

Alert:  there will be a lot of coding today!

**Note**:  In real life (production), you probably just use the existing tool such as `spaCy` (which we will learn).  Here we just learn so we can develop a deep understanding.

## 1. Parsing function

We gonna start with a class `Parsing`, representing a parser for each sentence.  For each sentence, we need the `stack`, `buffer`, and the `dependencies`.

In [3]:
sentence = ['He', 'has', 'good', 'control', '.']
something = sentence.pop(0)

In [4]:
something

'The'

In [5]:
sentence

['cat', 'sat']

In [7]:
#basically, it takes the current state of the buffer, stack, dependencies
#tell us how SHIFT, LA, RA changes these three objects

class Parsing(object):
    
    #init stack, buffer, dep
    def __init__(self, sentence):  
        self.sentence = sentence     #['The', 'cat', 'sat]  #conll format which is already in the tokenized form
        self.stack    = ['ROOT']
        self.buffer   = sentence[:]  #in the beginning, everything is inside the buffer
        self.dep      = []           #maintains a list of tuples of dep
    
    #parse function that tells me how shift, la, ra changes these three objects
    def parse_step(self, transition):     #transition could be either S, LA, RA
        if transition == 'S':
            #get the top guy in the buffer and put in stack
            head = self.buffer.pop(0)
            self.stack.append(head)
        elif transition == 'LA':  #stack = [ROOT, He, has] ==> append to dep (has, he) and then He is gone from the stack [ROOT, has]
            dependent = self.stack.pop(-2)  #He
            self.dep.append((self.stack[-1], dependent))  #(has, he)
        elif transition == 'RA':
            #can you guys try to this???
            dependent = self.stack.pop()  #stack = [ROOT, has, control] ==> dep (has, control), control will be gone fromt he stack [ROOT, has]
            self.dep.append((self.stack[-1], dependent))
        else:
            print(f"Bad transition: {transition}")
    
    #given some series of transition, it gonna for-loop the parse function
    def parse(self, transitions):
        for t in transitions:
            self.parse_step(t)
        return self.dep
    
    #check whether things are finished - no need to do anymore functions....
    def is_completed(self):
        return (len(self.buffer) == 0) and (len(self.stack) == 1)  #so buffer is empty and ROOT is the only guy in stack

### Testing the parse step

In [8]:
#create a instance of Parsing
parsing = Parsing(['He', 'has', 'good', 'control', '.'])

In [9]:
parsing.stack, parsing.buffer, parsing.dep

(['ROOT'], ['He', 'has', 'good', 'control', '.'], [])

In [10]:
#try to do a shift, and see what happen???
parsing.parse_step("S")
parsing.stack, parsing.buffer, parsing.dep

(['ROOT', 'He'], ['has', 'good', 'control', '.'], [])

In [12]:
#do shift, and then left arc
parsing.parse_step("S")
parsing.parse_step("LA")
parsing.stack, parsing.buffer, parsing.dep

(['ROOT', 'good'], ['control', '.'], [('has', 'He'), ('good', 'has')])

### Testing the parse

In [13]:
parsing = Parsing(['He', 'has', 'good', 'control', '.'])

#use the parse function, to tell it to do S, S, L, S, S, L, R
#double check whether we have three dep (has, he), (control, good), (has, control)
parsing = Parsing(["He", "has", "good", "control","."])
parsing.parse(["S","S", "LA", "S", "S", "LA", "RA"])
parsing.stack, parsing.buffer, parsing.dep

(['ROOT', 'has'],
 ['.'],
 [('has', 'He'), ('control', 'good'), ('has', 'control')])

### Minibatch parsing

We gonna create a minibatch loader that loads a bunch of sentences, and perform parse accordingly.  For now, we will assume a very dump model to predict the transitions.

In [ ]:
def minibatch_parse(sentences, model, batch_size):
    dep = []  #all the resulting dep
    
    #init Parsing instance for each sentence in the batch
    partial_parses = [Parsing(sentence) for sentence in sentences]  #in tokenized form
    #Parsing(['The', 'cat', 'sat']), Parsing(['Chaky', 'is', 'mad'])
    
    unfinished_parses = partial_parses[:]
    
    #while we still have sentence
    while unfinished_parses:  #if there are still a Parsing object
    
        #take a certain batch of sentence
        minibatch = unfinished_parses[:batch_size]
        
        #create a dummy model to tell us what's the next transition
        transitions = model.predict(minibatch) 
        
        #for transition predicted this dummy model
            #parse step
            
        #remove any sentence is finish
        
    #return dep

In [ ]:
class DummyModel(object):
    def predict(self, partial_parses):
        #partial_parses: list of Parsing instances
        #first shift everything onto the stack, and then just do RA if the first word
        #of the sentence is "right", otherwise, is "left"
        return [("RA" if pp.stack[1] == "right" else "LA") if len(pp.buffer) == 0 else "S"
                for pp in partial_parses] 

### Testing the minibatch

In [ ]:
sentences = [["right", "arcs", "only"],
             ["right", "arcs", "only", "again"],
             ["left", "arcs", "only"],
             ["left", "arcs", "only", "again"]]

minibatch_parse(sentences, DummyModel(), 2)

## 2. Load data

We used English Penn Treebank dataset in CoNLL format.

CoNLL is the conventional name for TSV formats in NLP (TSV - tab-separated values, i.e., CSV with <TAB> as separator).
It originates from a series of shared tasks organized at the Conferences of Natural Language Learning (hence the name)

In CoNLL formats,
- every word (token) is represented in one line
- every sentence is separated from the next by an empty line
- every column represents one annotation

There are many formats, in our case, our conll file has 10 columns, the important columns are:
- 1:  word
- 4:  pos
- 6:  head of the dependency
- 7:  type of dependency

### Testing the load function

### Testing the preprocessing data...

To understand, we can draw these in a dependency tree, with the help of spaCy.  **Note** that spaCy do not draw the ROOT for us, but imagine the head of "plays" is ROOT.

## 3. Parser

We gonna create a parser that gonna help us:
- create a `tok2id` dictionary in the `__init__` function
- numercalize `numercalize` the words, dependencies, and pos tags
- create training data, `create_instances` by leveraging the ground truth of the dependencies
- finally the `parse` function

This feature vector consists of a list of tokens. They can be represented as a list of integers $\mathbf{w} = [w_1, w_2, \cdots, w_m]$ where $m$ is the number of features and each $0 \leq w_i \leq |V|$ is the index of a token in the vocabulary ($|V|$ is the vocabulary size).  Then our network looks up an embedding for each word and concatenates them into a single input vector:

$$\mathbf{x} = [\mathbf{E}_{w_1}, \cdots, \mathbf{E}_{w_m}] \in \mathbb{R}^{dm}$$

where $\mathbf{E} \in \mathbb{R}^{|V| \times d}$ is an embedding matrix with each row $\mathbf{E}_w$ as the vector for a particular word $w$

#### Features

A total of 18 + 18 + 12 features were used in the paper.

- Feature 1: 18 features
  - (a). 6 - top 3 words on buffer, top 3 words on stack, 
  - (b). 6 - the first and second left most/rightmost children and the leftmost/rightmost of the **top word** (i.e., `stack[-1]`) on the stack,  - `(leftmost(0), rightmost(0), secondleftmost(0), secondrightmost(0), leftmost(leftmost(0)), rightmost(rightmost(0)))`
  - (c). 6 - the first and second left most/rightmost children and the leftmost/rightmost of the **second top word** (i.e., `stack[-2]`) on the stack, - `(leftmost(0), rightmost(0), secondleftmost(0), secondrightmost(0), leftmost(leftmost(0)), rightmost(rightmost(0)))`
- Feature 2: 18 pos - basically corresponding POS tags
- Feature 3: 12 dep - corresponding ARC, excluding 6 words on the stack/buffer..

**NOTE**: There is no 3a because each word itself does not have dependency

**NOTE**: For brevity, the table skipped 2b, 2c

**NOTE**: Here, in the dependency column, I used LABELED dependency.   But in my code, it is unlabeled, thus e.g., COMPOUND will not be needed.

| Stack | Buffer | Feature 1a | Feature 1b | Feature 1c | Feature 2a | Feature 3b | Feature 3c | Dependency (ARC) | Transition |
| :--   |  :--   | :--        | :--        | :--        | :--        | :--        | :--        | :--               | :--        |
| [ROOT] | [Ms., Haag, plays, Elianti, .] | [NULL, NULL, ROOT, Ms., Haag, plays] | [NULL, NULL, NULL, NULL, NULL, NULL] | [NULL, NULL, NULL, NULL, NULL, NULL] | [P-NULL, P-NULL, P-ROOT, PROPN, PROPN, VERB] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL]  | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL]| | Init |
| [ROOT, Ms.] | [Haag, plays, Elianti, .] | [NULL, ROOT, Ms., Haag, plays, Elianti] | [NULL, NULL, NULL, NULL, NULL, NULL]  | [NULL, NULL, NULL, NULL, NULL, NULL] | [P-NULL, P-ROOT, PROPN, PROPN, VERB, PROPN] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL]  | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL]| | SHIFT |
| [ROOT, Ms., Haag] | [plays, Elianti, .] | [ROOT, Ms., Haag, plays, Elianti, .] | [NULL, NULL, NULL, NULL, NULL, NULL]  | [NULL, NULL, NULL, NULL, NULL, NULL] | [P-ROOT, PROPN, PROPN, VERB, PROPN, PUNCT] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL]  | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL]|  | SHIFT |
| [ROOT, Haag] | [plays, Elianti, .] | [NULL, ROOT, Haag, plays, Elianti, .] | [Ms., NULL, NULL, NULL, NULL, NULL, NULL] | [NULL, NULL, NULL, NULL, NULL, NULL] | [P-NULL, P-ROOT, PROPN, VERB, PROPN, PUNCT] | [COMPOUND, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL] | (Haag, Ms., COMPOUND) | LEFTARC |
| [ROOT, Haag, plays] | [Elianti, .] | [ROOT, Haag, plays, Elianti, ., NULL] | [NULL, NULL, NULL, NULL, NULL, NULL] | [Ms., NULL, NULL, NULL, NULL, NULL] | [P-ROOT, PROPN, VERB, PROPN, PUNCT, P-NULL] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL] | [COMPOUND, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL] | (Haag, Ms., COMPOUND) | SHIFT |
| [ROOT, plays] | [Elianti, .] | [NULL, ROOT, plays, Elianti, ., NULL] | [Haag, NULL, NULL, NULL, Ms., NULL] | [NULL, NULL, NULL, NULL, NULL, NULL] | [P-NULL, P-ROOT, VERB, PROPN, PUNCT, P-NULL] | [NSUBJ, D-NULL, D-NULL, D-NULL, COMPOUND, D-NULL] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL] | (Haag, Ms., COMPOUND), (plays, Haag, NSUBJ) | LEFTARC |
| [ROOT, plays, Elianti] | [.] | [ROOT, plays, Elianti, ., NULL, NULL] | [NULL, NULL, NULL, NULL, NULL, NULL] | [Haag, NULL, NULL, NULL, Ms., NULL] | [P-ROOT, VERB, PROPN, PUNCT, P-NULL, P-NULL] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL] | [NSUBJ, D-NULL, D-NULL, D-NULL, COMPOUND, D-NULL] | (Haag, Ms., COMPOUND), (plays, Haag, NSUBJ) | SHIFT |
| [ROOT, plays] | [.] | [NULL, ROOT, plays, ., NULL, NULL] | [Haag, Elianti, NULL, NULL, Ms., NULL] | [NULL, NULL, NULL, NULL, NULL, NULL] | [P-NULL, P-ROOT, VERB, PUNCT, P-NULL, P-NULL] | [NSUBJ, DOBJ, D-NULL, D-NULL, COMPOUND, D-NULL] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL] | (Haag, Ms., COMPOUND), (plays, Haag, NSUBJ), (plays, Elianti, DOBJ) | RIGHTARC |
| [ROOT, plays, .] | [] | [ROOT, plays, ., NULL, NULL, NULL] | [NULL, NULL, NULL, NULL, NULL, NULL] | [Haag, Elianti, NULL, NULL, Ms., NULL] | [P-NULL, P-ROOT, VERB, PUNCT, P-NULL, P-NULL] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL] | [NSUBJ, DOBJ, D-NULL, D-NULL, COMPOUND, D-NULL] | (Haag, Ms., COMPOUND), (plays, Haag, NSUBJ), (plays, Elianti, DOBJ) | SHIFT |
| [ROOT, plays] | [] | [NULL, ROOT, plays, NULL, NULL, NULL] | [Haag, Elianti, NULL, plays, Ms., NULL] | [Haag, Elianti, NULL, NULL, Ms., NULL] | [P-NULL, P-ROOT, VERB, P-NULL, P-NULL, P-NULL] | [NSUBJ, PUNCT, D-NULL, DOBJ, COMPOUND, D-NULL] | [D-NULL, D-NULL, D-NULL, D-NULL, D-NULL, D-NULL] | (Haag, Ms., COMPOUND), (plays, Haag, NSUBJ), (plays, Elianti, DOBJ), (plays, ., PUNCT) | RIGHT-ARC |
| [ROOT] | [] |  | |  |  |  |  | (Haag, Ms., COMPOUND), (plays, Haag, NSUBJ), (plays, Elianti, DOBJ), (plays, ., PUNCT), (ROOT, plays, ROOT) | RIGHT-ARC |

In [ ]:
class ModelWrapper(object):
    def __init__(self, parser, dataset, sentence_id_to_idx):
        self.parser = parser
        self.dataset = dataset
        self.sentence_id_to_idx = sentence_id_to_idx

    def predict(self, partial_parses):
        mb_x = [self.parser.extract_features(p.stack, p.buffer, p.dep,
                                             self.dataset[self.sentence_id_to_idx[id(p.sentence)]])
                for p in partial_parses]
        mb_x = np.array(mb_x).astype('int32')
        mb_x = torch.from_numpy(mb_x).long()
        mb_l = [self.parser.legal_labels(p.stack, p.buffer) for p in partial_parses]

        pred = self.parser.model(mb_x)
        pred = pred.detach().numpy()
        
        #we need to multiply 10000 with legal labels, to force the model not to make any impossible prediction
        #other, when we parse sequentially, sometimes there is nothing in the buffer or stack, thus error....        
        pred = np.argmax(pred + 10000 * np.array(mb_l).astype('float32'), 1)
        pred = ["S" if p == 2 else ("LA" if p == 0 else "RA") for p in pred]
        
        return pred

In [ ]:
#a simple function to create ids.....
def build_dict(keys, offset=0):
    #keys = ['P_PREFIX:IN', 'P_PREFIX:DT', 'P_PREFIX:NNP', 'P_PREFIX:CD', so on...]
    #offset is needed because this tok2id has something already inside....
    count = Counter()
    for key in keys:
        count[key] += 1
    
    #most_common = [('P_PREFIX:NN', 70), ('P_PREFIX:IN', 57), ... , ('P_PREFIX:JJR', 1)]
    #we use most_common in case we only want some maximum pos tags....
    mc = count.most_common()
    
    #{'P_PREFIX:NN': 31, 'P_PREFIX:IN': 32, .., 'P_PREFIX:JJR': 62} 
    return {w[0]: index + offset for (index, w) in enumerate(mc)}

### Testing the parser `__init__`

### Testing the `numericalize`

## 4. Word Embedding

Word embedding length of 50.  In the paper, they applied a custom 50-embedding to all the words, pos, and dependencies.  For pos and dependencies, they claimed that there are some similarities that can be learned as well.

In [ ]:
print("4. Loading pretrained embeddings...",)
config = Config()
start = time.time()
word_vectors = {}
for line in open(config.embedding_file).readlines():
    we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
    word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

## 5. Preprocessing

In [ ]:
print("5. Preprocessing training data...",)
start = time.time()
train_examples = parser.create_instances(train_set)
print("took {:.2f} seconds".format(time.time() - start))

## 6. Minibatch loader

In [ ]:
def get_minibatches(data, minibatch_size, shuffle=True):
    data_size = len(data[0])
    indices = np.arange(data_size)
    if shuffle:
        np.random.shuffle(indices)
    for minibatch_start in np.arange(0, data_size, minibatch_size):
        minibatch_indices = indices[minibatch_start:minibatch_start + minibatch_size]
        yield [_minibatch(d, minibatch_indices) for d in data]

def _minibatch(data, minibatch_idx):
    return data[minibatch_idx] if type(data) is np.ndarray else [data[i] for i in minibatch_idx]

def minibatches(data, batch_size):
    x = np.array([d[0] for d in data])
    y = np.array([d[2] for d in data])
    one_hot = np.zeros((y.size, 3))
    one_hot[np.arange(y.size), y] = 1
    return get_minibatches([x, one_hot], batch_size)

### Testing your minibatch loader

In [ ]:
# for i, (train_x, train_y) in enumerate(minibatches(train_examples, 1024)):
#     print(train_x.shape)  #batch size, features
#     print(train_y.shape)        #one hot encoding of 3 actions - shift, la, ra

## 7. Neural Network

Let's train a neural network to predict, given the state of the stack, buffer, and dependencies, which transition should be applied next.

Recall that our input vector is:

$$\mathbf{x} = [\mathbf{E}_{w_1}, \cdots, \mathbf{E}_{w_m}] \in \mathbb{R}^{dm}$$

where $\mathbf{E} \in \mathbb{R}^{|V| \times d}$ is an embedding matrix with each row $\mathbf{E}_w$ as the vector for a particular word $w$

We then compute our prediction as:

$$\mathbf{h} = \text{ReLU}(\mathbf{xW} + \mathbf{b}_1)$$
$$\mathbf{l} = \mathbf{hU} + \mathbf{b}_2$$
$$\hat{\mathbf{y}} = \text{softmax}(l)$$

where $\mathbf{h}$ is referred to as the hidden layer, $\mathbf{l}$ is the logits, $\hat{\mathbf{y}}$ is the predictions, and $\text{ReLU}(z) = \text{max}(z, 0))$.  We will then train the model to minimize cross-entropy (CE) loss:

$$J(\theta) = \text{CE}(\mathbf{y}, \hat{\mathbf{y}}) = -\sum_{i=1}^{3}y_i \log \hat{y}_i$$

To compute the loss for the training set, we average this $J(\theta)$ across all training examples.  We will use UAS (Unlabeled Attachment Score) as main metric, which is computed as the ratio between number of correctly predicted dependencies and the number of total dependencies despite of the relations (which our model doesn't predict the relation since is unlabeled). 

Now let's code the <code>train_for_epoch</code> and <code>train</code> functions to actually train the model.

In [ ]:
#just a class to get the average.....
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train(parser, train_data, dev_data, output_path, batch_size=1024, n_epochs=10, lr=0.0005):
    
    best_dev_UAS = 0
    
    optimizer = optim.Adam(parser.model.parameters(), lr=0.001)
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(n_epochs):
        print("Epoch {:} out of {:}".format(epoch + 1, n_epochs))
        dev_UAS = train_for_epoch(
            parser, train_data, dev_data, optimizer, loss_func, batch_size)
        if dev_UAS > best_dev_UAS:
            best_dev_UAS = dev_UAS
            print("New best dev UAS! Saving model.")
            torch.save(parser.model.state_dict(), output_path)
        print("")


def train_for_epoch(parser, train_data, dev_data, optimizer, loss_func, batch_size):
    
    parser.model.train()  # Places model in "train" mode, i.e. apply dropout layer
    n_minibatches = math.ceil(len(train_data) / batch_size)
    loss_meter = AverageMeter()

    with tqdm(total=(n_minibatches)) as prog:
        for i, (train_x, train_y) in enumerate(minibatches(train_data, batch_size)):
            
            #train_x:  batch_size, n_features
            #train_y:  batch_size, target(=3)
            
            optimizer.zero_grad() 
            loss = 0.
            train_x = torch.from_numpy(train_x).long()  #long() for int so embedding works....
            train_y = torch.from_numpy(train_y.nonzero()[1]).long()  #get the index with 1 because torch expects label to be single integer

            # Forward pass: compute predicted logits.
            logits = parser.model(train_x)
            # Compute loss
            loss = loss_func(logits, train_y)
            # Compute gradients of the loss w.r.t model parameters.
            loss.backward()
            # Take step with optimizer.
            optimizer.step()

            prog.update(1)
            loss_meter.update(loss.item())

    print("Average Train Loss: {}".format(loss_meter.avg))
    print("Evaluating on dev set",)
    parser.model.eval()  # Places model in "eval" mode, i.e. don't apply dropout layer
        
    dev_UAS, _ = parser.parse(dev_data)
    print("- dev UAS: {:.2f}".format(dev_UAS * 100.0))
    return dev_UAS

## 8. Training

In [ ]:
#create directory if it does not exist for saving the weights...
output_dir = "output/{:%Y%m%d_%H%M%S}/".format(datetime.now())
output_path = output_dir + "model.weights"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005)

## 9. Testing

In [ ]:
print(80 * "=")
print("TESTING")
print(80 * "=")

print("Restoring the best model weights found on the dev set")
parser.model.load_state_dict(torch.load(output_path))
print("Final evaluation on test set",)
parser.model.eval()
UAS, dependencies = parser.parse(test_set)
print("- test UAS: {:.2f}".format(UAS * 100.0))
print("Done!")